# Artificial Neural Network

Run the following code only once (remove #) if tensorflow is not already installed

In [ ]:
#%pip install tensorflow

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
tf.random.set_seed(1)

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
df = pd.read_csv('diabetes.csv')
X = df.drop('Outcome',axis=1)
y = df['Outcome']

### Splitting the dataset into the Training set and Test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2,stratify=y)

### Feature Scaling

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")
y_pred

### Making the Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

# Parameter Tuning

Following function lets you define your own dynamic NN model with varying number of hidden layers, activation functions and number of units.

In [ ]:
def create_model(n_layers,activation,units):
    # create model
    model = tf.keras.models.Sequential()
    for i in range(0,n_layers):
        model.add(tf.keras.layers.Dense(units=units, activation=activation))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

You can use Sequential Keras models as part of your Scikit-Learn workflow via the wrappers

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
param_grid = {'n_layers':[1,2,3,4,5],'activation': ['relu','tanh'], 'units': [8,16,32,64]}

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=32, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,
                    verbose = 1,scoring='accuracy',return_train_score=True)
grid_result = grid.fit(X_train, y_train)

In [ ]:
params = grid_result.cv_results_['params']
means = grid_result.cv_results_['mean_test_score']
meants = grid_result.cv_results_['mean_train_score']
for meant, mean, param in zip(meants, means, params):
    print("Training %f and Validation %f with: %r" % (meant, mean, param))

In [ ]:
print(grid.best_params_)

In [ ]:
y_pred = (grid.predict(X_test)> 0.5).astype("int32")
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

# Regression with ANN

In [ ]:
df = pd.read_csv('Hitters_Data.csv')
df=df.dropna()
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = np.log(df.Salary)

# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')

# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis = 1)

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
def create_model(n_layers,activation,units,dropout):
    # create model
    model = tf.keras.models.Sequential()
#    model.add(tf.keras.layers.Dense(units=units, activation=activation))
    for i in range(0,n_layers-1):
#        model.add(tf.keras.layers.Dropout(dropout))
        model.add(tf.keras.layers.Dense(units=units, activation=activation))
    model.add(tf.keras.layers.Dense(units=1, activation='linear'))    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
#param_grid = {'n_layers':[1,2,3,4,5],'activation': ['relu'], 'units': [40,50,60],'dropout':[0.2,0.5,0.8]}
param_grid = {'n_layers':[1,2,3,4,5],'activation': ['linear','relu','tanh'], 'units': [10,40,50,60]}

In [ ]:
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=10)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5,
                    verbose = 1,scoring='neg_mean_squared_error',return_train_score=True)
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
params = grid_result.cv_results_['params']
means = grid_result.cv_results_['mean_test_score']
meants = grid_result.cv_results_['mean_train_score']
for meant, mean, param in zip(meants, means, params):
    print("Training %f and Validation %f with: %r" % (meant, mean, param))

In [ ]:
tf.random.set_seed(1)
model = create_model(5,'tanh',40)
model.fit(X_train, y_train, batch_size = 32, epochs = 100)
y_pred=model.predict(X_test)
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))